Test2　　全結合層(linear,relu)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [3]:
%cd "ctorch210701/MiniCTorch_Prototype"

/content/drive/My Drive/Colab Notebooks/ctorch210701/MiniCTorch_Prototype


In [4]:
! pip install lark-parser

     |████████████████████████████████| 235kB 25.0MB/s 
  Created wheel for lark-parser: filename=lark_parser-0.11.3-py2.py3-none-any.whl size=99739 sha256=fccb67d70e2b2ca2ebf57f537bd279febccbe9aae2bfaba0ddbe5462c9f746c8
  Stored in directory: /root/.cache/pip/wheels/81/4d/b0/a94848d29b8ea206b0ef798ad241e6ebbcbc51825bec7832d3
Successfully built lark-parser


In [5]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch
import minictorch.generator as GN
import minictorch.converter as CV

In [6]:
class Net( nn.Module ):
   def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(2, 32)
      self.fc2 = nn.Linear(32, 32)
      self.fc3 = nn.Linear(32, 3)

   def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
      #return F.log_softmax(x, dim = 1)


In [7]:
def generate_json( project, input_x, json_path, check=1 ): 
   
    model = Net()
    #print("weight", model.fc1.weight)
    #print("bias", model.fc1.bias)
    print(input_x)

    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_to_model, json_path )

    if check > 0:
        #torch.set_grad_enabled(True)  #　効かない(?)
        yy = model( input_x )
        print(yy)
        yy.sum().backward()
        print(input_x.grad)

    return model

In [8]:
def convert_json( project, model, input_x, json_path, rand_flag=0 ):

    folder = "src"
    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.cpp"
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"

    # load json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load( fp )

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len(code1) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )
    else:
       param_fname=""

    # save cpp file
    print( "[CPP]  ", cpp_path )
    code2 = CV.c_code_generator( obj, model, rand_flag ) # 210719 mod

    #print("[SAVE]",args.path+"/"+args.output)
    #ofp= open( args.path + "/" + args.output, "w" )
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    print( "[MAKE] ", make_path )
    make_code = CV.makefile_generator( project )  #210719 mod

    #makefp= open( args.path + "/" + "Makefile", "w" )
    makefp = open( make_path, "w" )
    makefp.write( make_code )


In [9]:
input_to_model = torch.randn((1,2),requires_grad=True)
project = "test2"

#torch.set_grad_enabled(True)
json_fname = project + ".json"
json_path  = "network/" + json_fname
model = generate_json( project, input_to_model, json_path )

tensor([[-0.4999, -0.8633]], requires_grad=True)
[JSON] network/test2.json
skip: Net/Linear[fc1]/weight/29
skip: Net/Linear[fc1]/weight/29
skip: Net/Linear[fc2]/weight/32
skip: Net/Linear[fc2]/weight/32
skip: Net/Linear[fc3]/weight/35
skip: Net/Linear[fc3]/weight/35
tensor([[0.1042, 0.2093, 0.0272]], grad_fn=<AddmmBackward>)
tensor([[-0.1689, -0.0215]])


In [10]:
 convert_json( project, model, input_to_model, json_path, 0 )

[JSON] network/test2.json
{'name': 'Net/Linear[fc1]/weight/43', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 1}
{'name': 'Net/Linear[fc1]/bias/42', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 2}
{'name': 'Net/Linear[fc2]/weight/46', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [7], 'sorted_id': 5}
{'name': 'Net/Linear[fc2]/bias/45', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [7], 'sorted_id': 6}
{'name': 'Net/Linear[fc3]/weight/49', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [11], 'sorted_id': 9}
{'name': 'Net/Linear[fc3]/bias/48', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [11], 'sorted_id': 10}
[PARAM] src/test2_param.cpp
[CPP]   src/test2.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'shape': [1, 2], 'out': [3], 'sorted_id': 0}
{'name': 'Net/Linear[fc1]/weight/43', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 1}
Net/Linear[fc1]/weight/43  ->  fc1_weight
{'name': 

In [11]:
!g++ -std=c++14 ./src/test2.cpp ./src/test2_param.cpp -I ../../ctorch/lib -lcblas -o test2

(注意)  ctorch/libにxtensor関連のincludeを置いています。

In [12]:
!./test2

### forward computation ...
{{ 0.104231,  0.209332,  0.027211}}
### backward computation ...
{{-0.168934, -0.021453}}
